In [ ]:
!pip install gradio

     |████████████████████████████████| 2.9 MB 4.4 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 212 kB 56.4 MB/s 
     |████████████████████████████████| 2.0 MB 39.9 MB/s 
     |████████████████████████████████| 253 kB 44.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 1.1 MB 36.3 MB/s 
     |████████████████████████████████| 271 kB 42.5 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
     |████████████████████████████████| 144 kB 40.3 MB/s 
     |████████████████████████████████| 10.9 MB 20.1 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 4.0 MB 20.3 MB/s 
     |████████████████████████████████| 61 kB 255 kB/s 
     |████████████████████████

In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import math
import pandas as pd
import numpy as np
from IPython.display import display

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


import time

from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import r2_score

/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
def greet(file_obj):
  df = pd.read_csv(file_obj.name)
  df = df.drop(['Date', 'High', 'Low', 'Adj Close'], axis = 'columns')
  df = df.drop(df.index[0:1321], axis = 0)

  def normalize_data(data):

    scaler = MinMaxScaler()
    numerical = ['Open', 'Close', 'Volume']
    data[numerical] = scaler.fit_transform(data[numerical])

    return data

  df = normalize_data(df)

  df = df.dropna(how='any',axis=0) 

  def train_test_split_lstm(stocks, prediction_time=1, test_data_size=450, unroll_length=50):
    test_data_cut = test_data_size + unroll_length + 1

    x_train = stocks[0:-prediction_time - test_data_cut].values
    y_train = stocks[prediction_time:-test_data_cut]['Close'].values

    x_test = stocks[0 - test_data_cut:-prediction_time].values
    y_test = stocks[prediction_time - test_data_cut:]['Close'].values

    return x_train, x_test, y_train, y_test

  X_train, X_test,y_train, y_test = train_test_split_lstm(df, 5)

  def unroll(data, sequence_length=24):

    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)

  unroll_length = 50
  X_train = unroll(X_train, unroll_length)
  X_test = unroll(X_test, unroll_length)
  y_train = y_train[-X_train.shape[0]:]
  y_test = y_test[-X_test.shape[0]:]

  def build_basic_model(input_dim, output_dim, return_sequences):
    model = Sequential()
    model.add(LSTM(
        input_shape=(None, input_dim),
        units=output_dim,
        return_sequences=return_sequences))

    model.add(LSTM(
        100,
        return_sequences=False))

    model.add(Dense(
        units=1))
    model.add(Activation('linear'))

    return model

  model = build_basic_model(input_dim = X_train.shape[-1],output_dim = unroll_length, return_sequences=True)

  start = time.time()
  model.compile(loss='mean_squared_error', optimizer='adam')

  def standardize(train, test):

    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

  standardize(X_train, X_test)

  model.fit(X_train, y_train, epochs=1, validation_split=0.05)

  predictions = model.predict(X_test)

    # plt.rcParams['figure.figsize'] = (16, 10)
  
    # ax = fig.add_subplot(111)

  fig = plt.figure()

  plt.plot(y_test, '#00FF00', label='Adjusted Close')
  plt.plot(predictions, '#0000FF', label='Predicted Close')

  plt.ylabel("Price in Rupees")
  plt.xlabel("Trading Days")

  plt.legend()

  # ax.set_title("Actual vs Predicted Stock Price")
    # ax.legend(loc='upper left')

  trainScore = model.evaluate(X_train, y_train, verbose=0)
  trainScorePrint = 'Train Score: %.8f MSE (%.8f RMSE)' % (trainScore, math.sqrt(trainScore))

  testScore = model.evaluate(X_test, y_test, verbose=0)
  testScorePrint = 'Test Score: %.8f MSE (%.8f RMSE)' % (testScore, math.sqrt(testScore))

  # explained_variance_score = explained_variance_score(y_test, predictions)
  # explained_variance_scorePrint = 'Explained Variance Score: %.8f EVS' % (explained_variance_score)

  # max_error = max_error(y_test, predictions)

  # r2_score = r2_score(y_test, predictions)

  return fig, trainScorePrint, testScorePrint

gr.Interface(greet, "file", [gr.outputs.Plot(type="auto"), "text", "text"]).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17542.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7ff7f9ef5b10>,
 'http://127.0.0.1:7861/',
 'https://17542.gradio.app')